## Proceso de Scrapping de Narrativas


In [2]:
import asyncio
from twscrape import API, gather
from twscrape.logger import set_log_level
import pandas as pd
import datetime
import os
from dotenv import load_dotenv,dotenv_values


In [4]:
# creo api para hacer las llamadas
api = API()

### Defino funciones

In [10]:
def save_info(dataset,filename):
    df = pd.DataFrame(dataset)
    df.to_csv(filename, index=False)

In [11]:
def read_and_concat(ruta,data):
    df_new = pd.DataFrame(data)
    df = pd.read_csv(ruta)
    df_actualizado = pd.concat([df, df_new], ignore_index=True)
    df_actualizado.to_csv(ruta, index=False)

In [12]:
def try_read_and_concat(ruta,data):
    df_new = pd.DataFrame(data)
    try: 
        df = pd.read_csv(ruta)
        df_actualizado = pd.concat([df, df_new], ignore_index=True)
        df_actualizado.to_csv(ruta, index=False)
    except:
        df = pd.DataFrame(data)
        df.to_csv(ruta, index=False)
   

In [15]:
async def get_top_tweets(keywords,limit=2000):
    tweets = await gather(api.search(keywords, 2000, kv={"product": "Top"}))
    return tweets

In [16]:
def parse_narrative_tweets(tweets,narrativa="",categoria="Tweet"):
    parsed_tweets = []
    parsed_users = []
    existing_ids = []
    for tweet in tweets:
            data_tweets = {
                    'Tweet id': tweet.id,
                    'Contenido': tweet.rawContent,
                    'Fecha': tweet.date,
                    'Categoria': categoria,
                    'Autor username': tweet.user.username,
                    'Autor displayname': tweet.user.displayname,
                    'Autor location': tweet.user.location,
                    'Link': tweet.url,
                    'Likes': tweet.likeCount,
                    'Respuestas': tweet.replyCount,
                    'Retweets': tweet.retweetCount,
                    'Hashtags': tweet.hashtags,
                    'Mentioned users': tweet.mentionedUsers,
                    'Narrativa':narrativa,
                    'Lang': tweet.lang
                }
            parsed_tweets.append(data_tweets)

            data_user = {
                'User id': tweet.user.id,
                'Autor username': tweet.user.username,
                'Autor displayname': tweet.user.displayname,
                'Autor location': tweet.user.location,
                'Fecha Creacion': tweet.user.created,
                'Link': tweet.user.url,
                'Verified':tweet.user.verified,
                'Followers': tweet.user.followersCount,
                'Following': tweet.user.friendsCount,
                'Tweets': tweet.user.statusesCount,
            }

            if tweet.user.id not in existing_ids:
                parsed_users.append(data_user)
                existing_ids.append(tweet.user.id)


    return parsed_tweets,parsed_users

### Obtengo tweets por narrativa

In [35]:
top_fraude = await get_top_tweets("Argentina Fraude elecciones",limit=2000)


In [36]:
len(top_fraude)

188

In [54]:
t1,t2 = parse_narrative_tweets(top_fraude,"Fraude")

### Obtengo narrativas


In [17]:
async def scrape_and_save_narratives(keywords,narrativa,ruta_tweets='data/narrativas/tweets.csv',ruta_users='data/narrativas/users.csv',limit=2000):
    tweets= await get_top_tweets(keywords,limit)
    print("Se obtuvieron " + str(len(tweets)) + " Tweets de los keywords: " + narrativa)
    
    parsed_tweets,parsed_user = parse_narrative_tweets(tweets,narrativa)
    print("Se parsearon correctamente los tweets")

    try_read_and_concat(ruta_tweets,parsed_tweets)
    try_read_and_concat(ruta_users,parsed_user)
    print("Se almaceno correctamente")

In [60]:
await scrape_and_save_narratives("Argentina Fraude elecciones",narrativa="Fraude")


Se obtuvieron 194 Tweets de los keywords: Fraude
Se parsearon correctamente los tweets
Se almaceno correctamente


In [ ]:
#await scrape_and_save_narratives("Argentina Fraude elecciones",narrativa="Fraude")

In [4]:
df = pd.read_csv("data/narrativas/tweets.csv")
df.shape

(194, 14)

In [5]:
df.head()

,Tweet id,Contenido,Fecha,Categoria,Autor username,Autor displayname,Autor location,Link,Likes,Respuestas,Retweets,Hashtags,Mentioned users,Narrativa
0,1717742780619845812,Argentina … más de siete nefastas décadas de p...,2023-10-27 03:19:27+00:00,Tweet,peladoMariano72,Aguante Militar 🇦🇷😸🦅🦆🦁🦍🚁🇸🇮,Argentina,https://twitter.com/peladoMariano72/status/171...,204,43,111,[],[],Fraude
1,1716824724876280133,ARGENTINA...\n\nASÍ ES COMO HICIERON EL FRAUDE...,2023-10-24 14:31:26+00:00,Tweet,deleon_efe,Efe Deleon,NaN,https://twitter.com/deleon_efe/status/17168247...,85,4,90,[],"[UserRef(id=1679055949, username='ALIENEGRO', ...",Fraude
2,1726251215661457845,🚨| URGENTE: Alertan de roturas a las boletas e...,2023-11-19 14:48:56+00:00,Tweet,eduardomenoni,Eduardo Menoni,Colombia/USA,https://twitter.com/eduardomenoni/status/17262...,4367,270,3299,[],[],Fraude
3,1716144908614918322,El gobierno de Argentina ya retrasó los result...,2023-10-22 17:30:05+00:00,Tweet,eduardomenoni,Eduardo Menoni,Colombia/USA,https://twitter.com/eduardomenoni/status/17161...,9479,345,3509,[],[],Fraude
4,1716249057004286328,🔴Lo que esta pasando en Argentina es lo mismo ...,2023-10-23 00:23:56+00:00,Tweet,medixi,MEDIXI,"Concepción, Chile",https://twitter.com/medixi/status/171624905700...,810,64,446,[],[],Fraude


In [6]:
df['Narrativa'].value_counts()

Narrativa
Fraude    194
Name: count, dtype: int64

In [7]:
autors = df['Autor username'].value_counts()


autors.head(n=20)


Autor username
eduardomenoni     4
fedetiberti       2
RDiosmel2         2
Karblack19        2
AnibalGarzon      2
carolina_moine    2
OldArg1810        2
evoespueblo       2
coralai_          2
ArgDiegote        2
CoralAINews       2
BizAnyio          2
alsersocimo       1
heroudr           1
ABCDigital        1
Jhoanromerofv     1
ARGprovincia25    1
lorstar08         1
MODO_Arg          1
ixXFelixXxi       1
Name: count, dtype: int64

### Analizo las narrativas